In [1]:
import os, sys, pprint
from azureml.core import Workspace, Image
from azureml.core.compute import ComputeTarget
from azureml.contrib.core.compute import IotHubCompute
from azureml.contrib.core.webservice import IotWebservice, IotBaseModuleSettings
from azureml.accel import AccelContainerImage

In [4]:
ws = Workspace.from_config()
iothub_compute = IotHubCompute(ws, 'fermi-edge')
image = Image(ws, 'im-klijnsma-tquarkrn50-v200') 

/uscms_data/d3/klijnsma/miniconda3/envs/azure0/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
print(iothub_compute)
print(image)

IotHubCompute(workspace=Workspace.create(name='Fermilab', subscription_id='80defacd-509e-410c-9812-6e52ed6a0016', resource_group='CMS_FPGA_Resources'), name=fermi-edge, id=/subscriptions/80defacd-509e-410c-9812-6e52ed6a0016/resourceGroups/CMS_FPGA_Resources/providers/Microsoft.MachineLearningServices/workspaces/Fermilab/computes/fermi-edge, type=IotHub, provisioning_state=Succeeded, location=eastus2, tags=None)
AccelContainerImage(workspace=Workspace.create(name='Fermilab', subscription_id='80defacd-509e-410c-9812-6e52ed6a0016', resource_group='CMS_FPGA_Resources'), name=im-klijnsma-tquarkrn50-v200, id=im-klijnsma-tquarkrn50-v200:1, tags={}, properties={}, version=1)


In [10]:
compute_targets = ComputeTarget.list(ws)
for t in compute_targets: 
    if t.type == "IotHub":
        print("IotHub '{}' has provisioning state '{}'.".format(t.name, t.provisioning_state))

IotHub 'fermi-edge' has provisioning state 'Succeeded'.


In [7]:
module_name = "module-toprn50-v200-0"

container_config = (
    '{ '
    #'  "ExposedPorts": {'
    #'    "50051/tcp": {}'
    #'  },'
    '  "HostConfig": { '
    '    "Binds": [ '
    '      "/etc/hosts:/etc/hosts" '
    '    ], '
    '    "Privileged": true, '
    '    "Devices": [ '
    '      { '
    '        "PathOnHost": "/dev/catapult0", '
    '        "PathInContainer": "/dev/catapult0" '
    '      }, '
    '      { '
    '        "PathOnHost": "/dev/catapult1", '
    '        "PathInContainer": "/dev/catapult1" '
    '      } '
    '    ], '
    '    "PortBindings": { '
    '      "50051/tcp": [ '
    '        { '
    '          "HostPort": "50051" '
    '        } '
    '      ] '
    '    } '
    '  } '
    '}'
    )

routes = {
    "route": "FROM /messages/* INTO "
}

# Here, we define the Azure ML module with the container_config options above
aml_module = IotBaseModuleSettings(
    name = module_name,
    create_option = container_config
    )
aml_module

In [9]:
# Then we define the deployment manifest for our IoT Edge device with the aml_module and routes
deploy_config = IotWebservice.deploy_configuration(
    device_id = 'fermi-edge',
    routes = {'route': 'FROM /messages/* INTO '},
    aml_module = aml_module,
    )

iot_service = IotWebservice.deploy_from_image(
    ws,
    module_name,
    image,
    deploy_config,
    iothub_compute
    )
iot_service.wait_for_deployment()

/uscms_data/d3/klijnsma/miniconda3/envs/azure0/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: deploy_from_image has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  
Received bad response from Model Management Service:
Response Code: 400
Headers: {'Date': 'Fri, 24 Apr 2020 14:51:41 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-client-request-id': 'a9125cb8e0bf4757b4521e44a1b7c0e7', 'x-ms-client-session-id': '8b544146-67c6-401a-a175-058eb716a1eb', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"BadRequest","statusCode":400,"message":"The request is invalid","details":[{"code":"IotDeploymentF

WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 400
Headers: {'Date': 'Fri, 24 Apr 2020 14:51:41 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-client-request-id': 'a9125cb8e0bf4757b4521e44a1b7c0e7', 'x-ms-client-session-id': '8b544146-67c6-401a-a175-058eb716a1eb', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"BadRequest","statusCode":400,"message":"The request is invalid","details":[{"code":"IotDeploymentFailed","message":"HttpRequest error, may be caused by broken connectionString, error detail: Name or service not known"}]}'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from Model Management Service:\nResponse Code: 40